In [2]:
# search for and download files
from pathlib import Path
from pprint import pprint
from ost.s1 import download
from ost.helpers import vector
from ost import Sentinel1, Generic

# define the project directory
project_dir = Path.home() / "Documents" / "R" / "s14soil" / "src" / "sen1_ost" / "data"
project_dir.mkdir(parents=True, exist_ok=True)
print(str(project_dir))

# define aoi
aoi = vector.aoi_to_wkt("../../data/fieldshapes/test_shape2.geojson")

# define the start and end date
start = "2020-05-01"
end = "2020-05-31"

# create an OST Generic class instance
ost_generic = Generic(project_dir=project_dir, aoi=aoi, start=start, end=end)

# initialize the Sentinel1 class
ost_s1 = Sentinel1(
    project_dir=project_dir,
    aoi=aoi,
    start=start,
    end=end,
    product_type="GRD",
    beam_mode="IW",
    polarisation="*",
)

# search command
# added scihub username and pw in Project.py line 260
ost_s1.search()

/home/david/Documents/R/s14soil/src/sen1_ost/data
 INFO (15:30:01): Created project directory at /home/david/Documents/R/s14soil/src/sen1_ost/data
 INFO (15:30:01): Downloaded data will be stored in: /home/david/Documents/R/s14soil/src/sen1_ost/data/download.
 INFO (15:30:01): Inventory files will be stored in: /home/david/Documents/R/s14soil/src/sen1_ost/data/inventory.
 INFO (15:30:01): Processed data will be stored in: /home/david/Documents/R/s14soil/src/sen1_ost/data/processing.
 INFO (15:30:01): Using /home/david/Documents/R/s14soil/src/sen1_ost/data/temp as directory for temporary files.
 INFO (15:30:01): Created project directory at /home/david/Documents/R/s14soil/src/sen1_ost/data
 INFO (15:30:01): Downloaded data will be stored in: /home/david/Documents/R/s14soil/src/sen1_ost/data/download.
 INFO (15:30:01): Inventory files will be stored in: /home/david/Documents/R/s14soil/src/sen1_ost/data/inventory.
 INFO (15:30:01): Processed data will be stored in: /home/david/Documents/R

/home/david/Documents/R/s14soil/src/sen1_ost/.venv/lib/python3.8/site-packages/ost/s1/search.py:212: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geo_df = geo_df.append(gdf)


In [ ]:
# show orbits / plot inventory
%matplotlib inline
%pylab inline
pylab.rcParams["figure.figsize"] = (19, 19)

ost_s1.plot_inventory(ost_s1.inventory, transparency=0.1)

In [ ]:
# refine inventory
ost_s1.refine_inventory()

In [ ]:
# plot refined inventory
pylab.rcParams["figure.figsize"] = (19, 19)

# define key to access refined inventory
key = "ASCENDING_VVVH"

ost_s1.refined_inventory_dict[key]
ost_s1.plot_inventory(ost_s1.refined_inventory_dict[key], 0.1)

In [6]:
# download data from refined inventory, defined by key
key = "ASCENDING_VVVH"
ost_s1.download(ost_s1.refined_inventory_dict[key], mirror="2", concurrent=10, uname='langdav', pword='SoLaDaSon_160317')

 INFO (15:30:35): All scenes are ready for being processed.


In [7]:
# set ard parameters
ost_s1.ard_parameters["single_ARD"]["resolution"] = 10  # set output resolution to 100m
ost_s1.ard_parameters["single_ARD"]["remove_speckle"] = True  # apply a speckle filter
ost_s1.ard_parameters["single_ARD"]["to_db"] = True
# ost_s1.ard_parameters["single_ARD"]["to_tif"] = True
# ost_s1.ard_parameters["single_ARD"]["out_projection"] = 3857

AttributeError: 'Sentinel1' object has no attribute 'ard_parameters'

In [ ]:
#preprocessing
ost_s1.create_ard(
    infile=ost_s1.get_path(project_dir), 
    out_dir=project_dir,
    subset=aoi,
    overwrite=True)

In [ ]:
# convert img to tiff
import gdal
import os
from pathlib import Path
for u in ['Gamma0_VV_db','Gamma0_VH_db']:
    paths = list(Path(project_dir).rglob(str(u + '.img')))
    for i in paths:
        inpath = str(i)
        outpath = '/home/david/OST_Tutorials/out/' + u + '_' + os.path.split(os.path.split(os.path.split(i)[0])[0])[1] + '.tiff'
        ds = gdal.Open(inpath)
        ds = gdal.Translate(outpath, ds)
        ds = None

# %%bash
# gdal_translate -of GTiff /home/david/OST_Tutorials/Tutorial_1/15/20220509/20220509_15_bs.data/Gamma0_VH_db.img /home/david/OST_Tutorials/Tutorial_1/15/20220509/20220509_15_bs.data/Gamma0_VH_db.tiff

In [ ]:
# clean
# preprocessing
# restore download dir (as download was not done with OST)
download.restore_download_dir(output_dir, output_dir)


for i in range(len(products_df)):
    # convert to Sentinel1Scene to work with Open SAR Toolkit
    s1 = Sentinel1Scene(products_df.iloc[i][0])

    # set ARD (Analysis Ready Data) parameters
    # pprint(s1.ard_parameters.keys())
    # pprint(s1.ard_parameters)
    # pprint(s1.ard_parameters["single_ARD"])
    s1.ard_parameters["single_ARD"]["resolution"] = 10  # set output resolution to 100m
    s1.ard_parameters["single_ARD"]["remove_speckle"] = True  # apply a speckle filter
    s1.ard_parameters["single_ARD"]["to_db"] = True
    # s1.ard_parameters["single_ARD"]["to_tif"] = True
    # s1.ard_parameters["single_ARD"]["out_projection"] = 3857

    # define AOI
    aoi = vector.aoi_to_wkt('../../data/fieldshapes/test_shape2.geojson')

    # do preprocessing
    s1.create_ard(
        infile=s1.get_path(output_dir), 
        out_dir=output_dir,
        subset=aoi,
        overwrite=True)

    # create RGB color composite
    # s1.create_rgb(outfile=output_dir / f"{s1.start_date}.tiff")
    # %matplotlib inline
    # %pylab inline
    # pylab.rcParams["figure.figsize"] = (23, 23)
    # s1.visualise_rgb(shrink_factor=2)